In [1]:
import os
from librosa.core import load, stft, istft, magphase
from librosa.output import write_wav
from config import *
from concurrent.futures import ThreadPoolExecutor
from time import time
import asyncio
import numpy as np
from multiprocessing import cpu_count

## Downsample original wav files.
1. Load from 'corpus/' and resample to 8192Hz.
2. Save to 'corpus_resized/'
3. Speed up processing by using a `ThreadPoolExecutor` with `#CPU * 2` threads.

In [2]:
base = 'ccmixter'
dirs = list(os.walk('corpus/' + base))[0][1]

def downsample(input_path, output_path):
    wav, _ = load(input_path, sr=SAMPLE_RATE)
    write_wav(output_path, wav, SAMPLE_RATE, norm=True)
    print(f"Saving {output_path}")
    
if __name__ == '__main__':
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs)):
            target_dir = 'corpus_resized/{}_{:0>2d}/'.format(base, i+1)
            os.makedirs(target_dir, exist_ok=True)
            pool.submit(downsample, f'corpus/{base}/{dirs[i]}/mix.wav', target_dir + 'mix.wav')
            pool.submit(downsample, f'corpus/{base}/{dirs[i]}/source-01.wav', target_dir + 'inst.wav')
            pool.submit(downsample, f'corpus/{base}/{dirs[i]}/source-02.wav', target_dir + 'vocal.wav')

Saving corpus_resized/ccmixter_01/mix.wav
Saving corpus_resized/ccmixter_01/vocal.wav
Saving corpus_resized/ccmixter_01/inst.wav
Saving corpus_resized/ccmixter_02/vocal.wav
Saving corpus_resized/ccmixter_02/inst.wav
Saving corpus_resized/ccmixter_02/mix.wav
Saving corpus_resized/ccmixter_03/inst.wav
Saving corpus_resized/ccmixter_03/mix.wav
Saving corpus_resized/ccmixter_04/inst.wav
Saving corpus_resized/ccmixter_04/mix.wav
Saving corpus_resized/ccmixter_04/vocal.wav
Saving corpus_resized/ccmixter_06/mix.wav
Saving corpus_resized/ccmixter_03/vocal.wav
Saving corpus_resized/ccmixter_05/mix.wav
Saving corpus_resized/ccmixter_05/inst.wav
Saving corpus_resized/ccmixter_05/vocal.wav
Saving corpus_resized/ccmixter_06/inst.wav
Saving corpus_resized/ccmixter_06/vocal.wav
Saving corpus_resized/ccmixter_07/mix.wav
Saving corpus_resized/ccmixter_07/inst.wav
Saving corpus_resized/ccmixter_07/vocal.wav
Saving corpus_resized/ccmixter_08/mix.wav
Saving corpus_resized/ccmixter_08/inst.wav
Saving corpu

## Save wav files to npz
1. Load wave files from `corpus_resized`.
2. Apply Short-time Fourier transform (STFT) to audio trios
3. Apply normalization to magnitudes and save as npz dict in `numpy/`

In [3]:
def load_as_mag(file):
    wav, _ = load(file, sr=None)
    spectrogram = stft(wav, n_fft=WINDOW_SIZE, hop_length=HOP_LENGTH)
    mag, _ = magphase(spectrogram)
    return mag.astype(np.float32)

def save_to_npz(base, sample):
    nps = {}
    mix = load_as_mag(f'{base}/{sample}/mix.wav')
    vocal = load_as_mag(f'{base}/{sample}/vocal.wav')
    inst = load_as_mag(f'{base}/{sample}/inst.wav')
    
    mix_max = mix.max()
    mix_norm = mix / mix_max
    vocal_norm = vocal / mix_max
    inst_norm = inst / mix_max
    print(f"Saving {sample}")
    np.savez_compressed(f'numpy/{sample}.npz', mix=mix_norm, vocal=vocal_norm, inst=inst_norm)
    
if __name__ == '__main__':
    dirs = sorted(list(os.walk('corpus_resized'))[0][1])
    with ThreadPoolExecutor(max_workers=cpu_count() * 2) as pool:
        for i in range(len(dirs)):
            pool.submit(save_to_npz, 'corpus_resized', dirs[i])

Saving ccmixter_04
Saving ccmixter_07
Saving ccmixter_01
Saving ccmixter_06
Saving ccmixter_02
Saving ccmixter_05
Saving ccmixter_03
Saving ccmixter_08
Saving ccmixter_11
Saving ccmixter_09
Saving ccmixter_10
Saving ccmixter_13
Saving ccmixter_12
Saving ccmixter_15
Saving ccmixter_17
Saving ccmixter_14
Saving ccmixter_18
Saving ccmixter_16
Saving ccmixter_19
Saving ccmixter_20
Saving ccmixter_21
Saving ccmixter_23
Saving ccmixter_24
Saving ccmixter_25
Saving ccmixter_26
Saving ccmixter_22
Saving ccmixter_29
Saving ccmixter_27
Saving ccmixter_28
Saving ccmixter_31
Saving ccmixter_32
Saving ccmixter_33
Saving ccmixter_34
Saving ccmixter_30
Saving ccmixter_37
Saving ccmixter_36
Saving ccmixter_35
Saving ccmixter_40
Saving ccmixter_39
Saving ccmixter_41
Saving ccmixter_38
Saving ccmixter_42
Saving ccmixter_43
Saving ccmixter_45
Saving ccmixter_46
Saving ccmixter_44
Saving ccmixter_47
Saving ccmixter_48
Saving ccmixter_49
Saving ccmixter_50
